<a href="https://colab.research.google.com/github/imji0319/PDSH/blob/master/NYC_modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My Drive/data/nyc_split

/content/gdrive/My Drive/data/nyc_split


In [0]:
ls

nyc_0.feather   nyc_14.feather  nyc_19.feather  nyc_4.feather  nyc_9.feather
nyc_10.feather  nyc_15.feather  nyc_1.feather   nyc_5.feather
nyc_11.feather  nyc_16.feather  nyc_20.feather  nyc_6.feather
nyc_12.feather  nyc_17.feather  nyc_2.feather   nyc_7.feather
nyc_13.feather  nyc_18.feather  nyc_3.feather   nyc_8.feather


In [0]:
pip install -U pyarrow

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (0.13.0)


In [0]:
import pandas as pd
import numpy as np

In [0]:
# read data
df_train =pd.read_feather('nyc_0.feather')

In [0]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600000 entries, 0 to 2599999
Data columns (total 13 columns):
level_0               int64
index                 int64
fare_amount           float32
pickup_longitude      float32
pickup_latitude       float32
dropoff_longitude     float32
dropoff_latitude      float32
passenger_count       uint8
year                  int64
weekday               uint8
hour                  uint8
distance_miles        float32
distance_to_center    float32
dtypes: float32(7), int64(3), uint8(3)
memory usage: 136.4 MB


In [0]:
df_train.head()

,level_0,index,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,hour,distance_miles,distance_to_center
0,0,0,4.5,-73.844315,40.721317,-73.841614,40.712276,1,2009,0,17,0.640473,8.501885
1,1,1,16.9,-74.016045,40.711304,-73.979271,40.782005,1,2010,1,16,5.250587,0.543003
2,2,2,5.7,-73.982735,40.761269,-73.991241,40.750561,2,2011,3,0,0.863477,3.482071
3,3,3,7.7,-73.987129,40.733143,-73.991570,40.758091,1,2012,5,4,1.739349,1.654125
4,4,4,5.3,-73.968094,40.768009,-73.956657,40.783764,1,2010,1,7,1.242172,4.225934


In [0]:
df_train = df_train.drop(columns=['level_0', 'index'])

In [0]:
from sklearn.model_selection import train_test_split 
import xgboost as xgb

In [0]:
y = df_train['fare_amount']
train = df_train.drop(columns=['fare_amount'])

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, y, random_state = 100, test_size=0.25)

In [0]:
#Cross-validation
params = {
    # Parameters that we are going to tune.
    'max_depth': 8, #Result of tuning with CV
    'eta':.03, #Result of tuning with CV
    'subsample': 1, #Result of tuning with CV
    'colsample_bytree': 0.8, #Result of tuning with CV
    # Other parameters
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 1
}

#Block of code used for hypertuning parameters. Adapt to each round of parameter tuning.
#Turn off CV in submission
CV=False
if CV:
    dtrain = xgb.DMatrix(train,label=y)
    gridsearch_params = [
        (eta)
        for eta in np.arange(.04, 0.12, .02)
    ]

    # Define initial best params and RMSE
    min_rmse = float("Inf")
    best_params = None
    for (eta) in gridsearch_params:
        print("CV with eta={} ".format(
                                 eta))

        # Update our parameters
        params['eta'] = eta

        # Run CV
        cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=1000,
            nfold=3,
            metrics={'rmse'},
            early_stopping_rounds=10
        )

        # Update best RMSE
        mean_rmse = cv_results['test-rmse-mean'].min()
        boost_rounds = cv_results['test-rmse-mean'].argmin()
        
        print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
        if mean_rmse < min_rmse:
            min_rmse = mean_rmse
            best_params = (eta)

    print("Best params: {}, RMSE: {}".format(best_params, min_rmse))
    
else:
    #Print final params to use for the model
    params['silent'] = 0 #Turn on output
    print(params)

{'max_depth': 8, 'eta': 0.03, 'subsample': 1, 'colsample_bytree': 0.8, 'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': 0}


In [0]:
def XGBmodel(x_train,x_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(x_train,label=y_train)
    matrix_test = xgb.DMatrix(x_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=500, 
                    early_stopping_rounds=10,evals=[(matrix_test,'test')])
    return model

In [0]:
def XGBmodelusedW(x_train,x_test,y_train,y_test,params,weights):
    matrix_train = xgb.DMatrix(x_train,label=y_train)
    matrix_test = xgb.DMatrix(x_test,label=y_test)
    model=xgb.train(params=params, weight=weigths,
                    dtrain=matrix_train,num_boost_round=500, 
                    early_stopping_rounds=10,evals=[(matrix_test,'test')])
    return model

In [0]:
model = XGBmodel(X_train,X_test,y_train,y_test,params)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[02:56:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	test-rmse:13.9874
Will train until test-rmse hasn't improved in 10 rounds.
[1]	test-rmse:13.6006
[2]	test-rmse:13.2252
[3]	test-rmse:12.8607
[4]	test-rmse:12.5101
[5]	test-rmse:12.1865
[6]	test-rmse:11.855
[7]	test-rmse:11.5334
[8]	test-rmse:11.2231
[9]	test-rmse:10.9382
[10]	test-rmse:10.6487
[11]	test-rmse:10.3671
[12]	test-rmse:10.095
[13]	test-rmse:9.85238
[14]	test-rmse:9.59882
[15]	test-rmse:9.3522
[16]	test-rmse:9.11555
[17]	test-rmse:8.89993
[18]	test-rmse:8.67813
[19]	test-rmse:8.47563
[20]	test-rmse:8.28125
[21]	test-rmse:8.08143
[22]	test-rmse:7.90115
[23]	test-rmse:7.71493
[24]	test-rmse:7.53449
[25]	test-rmse:7.36072
[26]	test-rmse:7.19228
[27]	test-rmse:7.03308
[28]	test-rmse:6.87689
[29]	test-rmse:6.72645
[30]	test-rmse:6.5821
[31]	test-rmse:6.44329
[32]	test-rmse:6.31
[33]	test-rmse:6.18051
[34]	test-rmse:6.06512
[35]	test-rmse:5.94638
[36]	

In [0]:
import pickle
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
model.info()
model.shape

In [0]:
# save model to file
joblib.dump(model, "pima.joblib.dat")

# Test Prediction 

In [0]:
cd /content/gdrive/My Drive/data

/content/gdrive/My Drive/data


In [0]:
df_test = pd.read_csv('test.csv')

In [0]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [0]:
nyc = (-74.0063889, 40.7141667)

In [0]:
df_test['pickup_datetime'] =df_test['pickup_datetime'].astype('datetime64[ns]')
df_test['year'] = df_test.pickup_datetime.apply(lambda t: t.year)
df_test['weekday'] = df_test['pickup_datetime'].dt.dayofweek
df_test['hour'] = df_test.pickup_datetime.apply(lambda t: t.hour)


df_test['distance_miles'] =distance(df_test.pickup_latitude, df_test.pickup_longitude, \
                                                            df_test.dropoff_latitude,df_test.dropoff_longitude)

df_test['distance_to_center'] = distance(nyc[1],nyc[0], \
                                                            df_test.pickup_latitude, df_test.pickup_longitude)

In [0]:
df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,hour,distance_miles,distance_to_center
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,13,1.443607,3.841863
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,13,1.507044,1.084252
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44,-73.982524,40.751260,-73.979654,40.746139,1,2011,5,11,0.384398,2.851254
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12,-73.981160,40.767807,-73.990448,40.751635,1,2012,5,21,1.218529,3.934481
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12,-73.966046,40.789775,-73.988565,40.744427,1,2012,5,21,3.347514,5.634652


In [0]:
test_key = df_test['key']
x_pred = df_test.drop(columns=['key', 'pickup_datetime'])

In [0]:
#Predict from test set
prediction = model.predict(xgb.DMatrix(x_pred), ntree_limit = model.best_ntree_limit)

In [0]:
submission = pd.DataFrame({'key' : df_test.key,
                          'fare_amount': prediction},
                         columns =['key','fare_amount'])

submission.head(10)

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.901086
1,2015-01-27 13:08:24.0000003,11.187179
2,2011-10-08 11:53:44.0000002,4.599588
3,2012-12-01 21:12:12.0000002,7.782683
4,2012-12-01 21:12:12.0000003,13.843194
5,2012-12-01 21:12:12.0000005,9.382550
6,2011-10-06 12:10:20.0000001,4.996400
7,2011-10-06 12:10:20.0000003,48.945194
8,2011-10-06 12:10:20.0000002,11.299504
9,2014-02-18 15:22:20.0000002,6.938256


In [0]:
submission.to_csv('xgb_submission1.csv', index=False)